#  Neural Machine Translation (NMT) with Seq2Seq and Attention

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

In [2]:
# Sample English-French dataset
data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("good morning", "bonjour"),
    ("thank you", "merci"),
    ("good night", "bonne nuit"),
    ("see you later", "à bientôt"),
    ("i love you", "je t'aime"),
]

In [3]:
# Preprocess the data
input_texts, target_texts = zip(*data)
target_texts = ["\t" + text + "\n" for text in target_texts]  # Add start ('\t') and end ('\n') tokens

In [4]:
# Tokenize the input and target texts
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
input_vocab_size = len(input_tokenizer.word_index) + 1

target_tokenizer = Tokenizer(char_level=True)  # Use character-level tokenization for the target language
target_tokenizer.fit_on_texts(target_texts)
target_sequences = target_tokenizer.texts_to_sequences(target_texts)
target_vocab_size = len(target_tokenizer.word_index) + 1

In [5]:
# Determine the maximum sequence length for padding
max_encoder_seq_length = max(len(seq) for seq in input_sequences)
max_decoder_seq_length = max(len(seq) for seq in target_sequences)


In [6]:
# Pad the sequences
encoder_input_data = pad_sequences(input_sequences, maxlen=max_encoder_seq_length, padding="post")
decoder_input_data = pad_sequences(target_sequences, maxlen=max_decoder_seq_length, padding="post")


In [7]:
# Create decoder target data (one-hot encoded)
decoder_target_data = np.zeros(
    (len(data), max_decoder_seq_length, target_vocab_size), dtype="float32"
)
for i, seq in enumerate(target_sequences):
    for t, token in enumerate(seq[1:]):  # Skip the start token for target data
        decoder_target_data[i, t, token] = 1.0


In [8]:
# Build the Seq2Seq Model
# Encoder
encoder_inputs = Input(shape=(max_encoder_seq_length,))
encoder_embedding = Embedding(input_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_decoder_seq_length,))
decoder_embedding = Embedding(target_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)


In [9]:
# Compile the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [10]:
# Train the model
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.0133 - loss: 1.9063 - val_accuracy: 0.0000e+00 - val_loss: 2.0300
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.2000 - loss: 1.8941 - val_accuracy: 0.0667 - val_loss: 2.0285
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.1733 - loss: 1.8813 - val_accuracy: 0.1000 - val_loss: 2.0267
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.1867 - loss: 1.8671 - val_accuracy: 0.1000 - val_loss: 2.0246
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.1867 - loss: 1.8510 - val_accuracy: 0.1000 - val_loss: 2.0219
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.1867 - loss: 1.8320 - val_accuracy: 0.1000 - val_loss: 2.0184
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.1867 - loss: 1.8089 - val_accuracy: 0.1000 - val_loss: 2.0137
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.1867 - loss: 1.7804 - val_accuracy: 0.1000 - 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.3733 - loss: 0.7925 - val_accuracy: 0.1000 - val_loss: 2.2701
Epoch 52/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.3867 - loss: 0.7780 - val_accuracy: 0.1000 - val_loss: 2.2781
Epoch 53/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.4000 - loss: 0.7663 - val_accuracy: 0.0667 - val_loss: 2.2885
Epoch 54/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.4000 - loss: 0.7513 - val_accuracy: 0.0667 - val_loss: 2.2999
Epoch 55/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.4000 - loss: 0.7356 - val_accuracy: 0.1000 - val_loss: 2.3066
Epoch 56/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.4267 - loss: 0.7264 - val_accuracy: 0.1000 - val_loss: 2.3143
Epoch 57/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 0.4400 - loss: 0.7155 - val_accuracy: 0.1000 - val_loss: 2.3232
Epoch 58/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step - accuracy: 0.4667 - loss: 0.7034 - val_accuracy: 0.1000 - val_lo

In [11]:
# Create the Inference Models
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)


In [12]:
# Function to decode the sequence
def decode_sequence(input_seq):
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate the target sequence
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_tokenizer.word_index["\t"]

    # Sampling loop
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_tokenizer.index_word[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [13]:
# Test the model
for input_text in input_texts:
    input_seq = pad_sequences(
        input_tokenizer.texts_to_sequences([input_text]), maxlen=max_encoder_seq_length
    )
    decoded_sentence = decode_sequence(input_seq)
    print(f"Input: {input_text} -> Predicted translation: {decoded_sentence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Input: hello -> Predicted translation: bonjourr

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/ste